# Implementación del algoritmo de Newton-Raphson

## 0 - Aplicación de Newton-Raphson al sistema estudiado

### Paso 1 : Importación de las librerias

In [14]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
import pandapower as pp # type: ignore
import matplotlib.pyplot as plt # type: ignore 

### Paso 2: Declaración de las variables del sistema (impedencia, potencias activas y reactivas demandadas, tensiones y angulos de inicio)

In [15]:
# Crear red
net = pp.create_empty_network()

# Crear barras
barra1 = pp.create_bus(net, vn_kv=110, name="Barra 1")
barra2 = pp.create_bus(net, vn_kv=220, name="Barra 2")
barra1A = pp.create_bus(net, vn_kv=220, name="Barra 1A")
barra1B = pp.create_bus(net, vn_kv=220, name="Barra 1B")
barra2A = pp.create_bus(net, vn_kv=220, name="Barra 2A")
barra2B = pp.create_bus(net, vn_kv=220, name="Barra 2B")
barra3A = pp.create_bus(net, vn_kv=220, name="Barra 3A")

# Crear transformador
pp.create_transformer(net, barra2, barra1, std_type="100 MVA 220/110 kV")

# Crear barra slack
pp.create_ext_grid(net, barra1, vm_pu=1.0, name="Slack bus")

# Crear cargas
pp.create_load(net, barra1A, p_mw=30, q_mvar=20, name="Carga 1A")
pp.create_load(net, barra1B, p_mw=15, q_mvar=10, name="Carga 1B")
pp.create_load(net, barra2A, p_mw=52.5, q_mvar=35, name="Carga 2A")
pp.create_load(net, barra2B, p_mw=90, q_mvar=60, name="Carga 2B")
pp.create_load(net, barra3A, p_mw=22.5, q_mvar=15, name="Carga 3A")

# Crear líneas
pp.create_line(net, barra2, barra1A, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2-1A")
pp.create_line(net, barra1A, barra2A, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1A-2A")
pp.create_line(net, barra2A, barra3A, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2A-3A")
pp.create_line(net, barra3A, barra2B, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L3A-2B")
pp.create_line(net, barra2B, barra1B, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2B-1B")
pp.create_line(net, barra1B, barra2, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1-B2")

pp.runpp(net,numba=False)

#Potencias cargas nominal en cada barra
P= 150
Q= 100

#creacion Ybus
Ybus=net._ppc['internal']['Ybus']

Ybus_df = pd.DataFrame(data=Ybus.todense())
print("Matriz Ybus (7x7):")
print(Ybus_df)

Matriz Ybus (7x7):
                       0                           1  \
0  18.069305-833.143703j    -18.041806+  833.131714j   
1 -18.041806+833.131714j  28090.834926-45050.010168j   
2   0.000000+  0.000000j -14036.382810+22117.936550j   
3   0.000000+  0.000000j -14036.382810+22117.936550j   
4   0.000000+  0.000000j      0.000000+    0.000000j   
5   0.000000+  0.000000j      0.000000+    0.000000j   
6   0.000000+  0.000000j      0.000000+    0.000000j   

                            2                           3  \
0      0.000000+    0.000000j      0.000000+    0.000000j   
1 -14036.382810+22117.936550j -14036.382810+22117.936550j   
2  23393.971351-36839.469289j      0.000000+    0.000000j   
3      0.000000+    0.000000j  18715.177081-29452.568795j   
4  -9357.588540+14745.291033j      0.000000+    0.000000j   
5      0.000000+    0.000000j  -4678.794270+ 7372.645517j   
6      0.000000+    0.000000j      0.000000+    0.000000j   

                            4              

### Paso 3: Ejecución y resultados del algoritmo de Newton-Raphson

In [16]:
# Ejecutar el método de Newton-Raphson
# Vector de valores iniciales (V y θ)
V = net.res_bus.vm_pu.values
angulo = net.res_bus.va_degree.values * np.pi / 180
x = np.concatenate([V, angulo])

# Resolver el sistema
solucion = newton_raphson(Fun_pot, Jacobiano, x)
print("Solución:", solucion)

Convergencia alcanzada en 1 iteraciones.
Solución: [ 1.00000000e+00  1.03338010e+00  1.03145705e+00  1.03237463e+00
  1.02923741e+00  1.02743491e+00  1.02836460e+00  1.29142298e-18
 -2.49171429e-01 -2.52938397e-01 -2.52545089e-01 -2.57092859e-01
 -2.59591932e-01 -2.59442236e-01]


## 1 - Función de calculo de la jacobiana

In [17]:

def Jacobiano(x):
    V = x[:len(x)//2]
    theta = x[len(x)//2:]
    
    J11 = np.zeros((len(V), len(V)))
    J12 = np.zeros((len(V), len(V)))
    J21 = np.zeros((len(V), len(V)))
    J22 = np.zeros((len(V), len(V)))
    
    for i in range(len(V)):
        for j in range(len(V)):
            G = Ybus[i, j].real
            B = Ybus[i, j].imag
            
            if i == j:
                for k in range(len(V)):
                    if k != i:
                        J11[i, j] += V[i] * V[k] * (-G * np.sin(theta[i] - theta[k]) + B * np.cos(theta[i] - theta[k]))
                        J12[i, j] += V[k] * (G * np.cos(theta[i] - theta[k]) + B * np.sin(theta[i] - theta[k]))
                        J21[i, j] += V[i] * V[k] * (G * np.cos(theta[i] - theta[k]) + B * np.sin(theta[i] - theta[k]))
                        J22[i, j] += V[k] * (G * np.sin(theta[i] - theta[k]) - B * np.cos(theta[i] - theta[k]))
                J11[i, j] += V[i] * Ybus[i, i].imag
                J12[i, j] += V[i] * Ybus[i, i].real
                J21[i, j] += V[i] * (-Ybus[i, i].real)
                J22[i, j] += V[i] * Ybus[i, i].imag
            else:
                J11[i, j] = V[i] * V[j] * (G * np.sin(theta[i] - theta[j]) - B * np.cos(theta[i] - theta[j]))
                J12[i, j] = V[i] * (G * np.cos(theta[i] - theta[j]) + B * np.sin(theta[i] - theta[j]))
                J21[i, j] = V[i] * V[j] * (-G * np.cos(theta[i] - theta[j]) - B * np.sin(theta[i] - theta[j]))
                J22[i, j] = V[i] * (G * np.sin(theta[i] - theta[j]) - B * np.cos(theta[i] - theta[j]))
    
    return np.block([[J11, J12], [J21, J22]])


## 2 - Funcion de calculo de la diferencia entre potencia inyecta y potencia calculada (función de error)

In [18]:
def Fun_pot(x):
    V = x[:len(x)//2]
    theta = x[len(x)//2:]
    
    P = np.zeros(len(V))
    Q = np.zeros(len(V))
    
    for i in range(len(V)):
        for j in range(len(V)):
            G = Ybus[i, j].real
            B = Ybus[i, j].imag
            P[i] += V[i] * V[j] * (G * np.cos(theta[i] - theta[j]) + B * np.sin(theta[i] - theta[j]))
            Q[i] += V[i] * V[j] * (G * np.sin(theta[i] - theta[j]) - B * np.cos(theta[i] - theta[j]))
    
    P_spec = net.res_bus.p_mw.values / net.sn_mva
    Q_spec = net.res_bus.q_mvar.values / net.sn_mva

    # Excluir barra slack de las ecuaciones
    P[0] = 0
    Q[0] = 0
    P_spec[0] = 0
    Q_spec[0] = 0
    
    return np.concatenate([-P - P_spec, -Q - Q_spec])


## 3 - Funcion principal de iteraciones del algoritmo

In [19]:
def newton_raphson(F, J, x0, tol=1e-6, max_iter=100):
    x = x0
    for i in range(max_iter):
        Jx = J(x)
        Fx = F(x)
        delta = np.linalg.solve(Jx, -Fx)
        x = x + delta
        if np.linalg.norm(delta, np.inf) < tol:
            print(f'Convergencia alcanzada en {i+1} iteraciones.')
            break
    else:
        print('No se alcanzó la convergencia.')
    return x
